In [1]:
import sys
import os
sys.path.append(r"c:\Users\david\OneDrive\Desktop\Python\MH_decipher")
from kryptolib import *



In [2]:
# Knihovna kryptolib je již importována v předchozí buňce, není třeba znovu importovat.


In [3]:
%pip install numpy

# === Vytvoření bigramové a trigramové matice ze souboru krakatit.txt ===
import numpy as np

# 1️ Načtu text a nahradím mezery podtržítkem (standard v šifrování)
import os

if not os.path.exists("krakatit.txt"):
    print("❌ Soubor 'krakatit.txt' nebyl nalezen. Ujistěte se, že existuje ve správném adresáři.")
    ref_text = ""
else:
    with open("krakatit.txt", encoding="utf-8") as f:
        ref_text = f.read().upper().replace(" ", "_")

# 2️ Vyfiltruju jen znaky, které jsou v naší definované abecedě
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
ref_text_filtered = "".join([char for char in ref_text if char in alphabet])

# 3️ Vytvořím bigramovou tabulku pomocí numpy
n = len(alphabet)
tm = np.ones((n, n), dtype=int)
char_to_index = {char: i for i, char in enumerate(alphabet)}

for i in range(len(ref_text_filtered) - 1):
    c1, c2 = ref_text_filtered[i], ref_text_filtered[i + 1]
    if c1 in char_to_index and c2 in char_to_index:
        tm[char_to_index[c1], char_to_index[c2]] += 1

# 4️ Normalizace: přepočítám hodnoty na pravděpodobnosti
bigram_model_np = tm / tm.sum()

# 5️ Vytvoření trigramového modelu jako slovník
try:
    from MH_decipher.kryptolib import create_ngram_log_prob_model
    trigram_model_dict = create_ngram_log_prob_model(ref_text_filtered, 3)
except ModuleNotFoundError:
    print("❌ Modul 'MH_decipher' nebyl nalezen. Ujistěte se, že adresář MH_decipher je ve správné cestě nebo nainstalujte balíček.")
    trigram_model_dict = {}

print("Bigram model shape:", bigram_model_np.shape)
print("Trigram model keys:", list(trigram_model_dict.keys())[:10])


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


❌ Modul 'MH_decipher' nebyl nalezen. Ujistěte se, že adresář MH_decipher je ve správné cestě nebo nainstalujte balíček.
Bigram model shape: (27, 27)
Trigram model keys: []


In [4]:
# === Test: šifrování a dešifrování jednoho textu ===
key = generate_random_key()
text = "BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS"
encrypted = substitute_encrypt(text, key)
decrypted = substitute_decrypt(encrypted, key)
print("Encrypted:", encrypted)
print("Decrypted:", decrypted)



Encrypted: R_SJIZXTONJEKUKQJIQEONJFCMJEKUKQONJFCMJR_SJSCHW_JUCH
Decrypted: BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS


In [5]:
# Kontrola dešifrovaného výstupu
print("✅ Dešifrovaný text:", decrypted)


✅ Dešifrovaný text: BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS


In [6]:
def prolom_substitute_hybrid(encrypted, bigram_model, trigram_model, iter_count=20000):
    # Vytvořím prázdný seznam, do kterého budu ukládat všechny výsledky během průběhu algoritmu
    all_results = []

    # Začínám s nejhorším možným skóre, abych mohla později porovnávat a hledat lepší
    best_score = float('-inf')

    # Zatím nemám žádného „nejlepšího kandidáta“
    best_candidate = None

    # Hlavní cyklus – proběhne iter_count-krát (např. 20000 pokusů)
    for i in range(iter_count):
        # Vygeneruju náhodný klíč
        candidate_key = generate_random_key()

        # Pomocí tohoto klíče zkusím dešifrovat text
        decrypted_candidate = substitute_decrypt(encrypted, candidate_key)

        # Vyhodnotím, jak „dobrý“ je dešifrovaný text – čím více platných slov, tím lépe
        p_candidate = score_by_dictionary(decrypted_candidate, WORDLIST_SET)

        # Uložím výsledek tohoto pokusu (klíč, dešifrovaný text, skóre)
        all_results.append({
            'key': candidate_key,
            'plaintext': decrypted_candidate,
            'score': p_candidate
        })

        # Pokud tento výsledek má lepší skóre než dosavadní nejlepší, tak si ho zapamatuju
        if p_candidate > best_score:
            best_score = p_candidate
            best_candidate = {
                'key': candidate_key,
                'plaintext': decrypted_candidate,
                'score': p_candidate
            }

    # Vrátím nejlepší nalezený výsledek a celý seznam všech pokusů (pro pozdější analýzu nebo uložení)
    return best_candidate, all_results

### Shrnutí

- Šifrování a dešifrování textu pomocí substituční šifry
- Automatické prolomení šifry pomocí Metropolis-Hastings algoritmu
- Použití bigramové a trigramové analýzy pro hodnocení textu
- Referenční text: Krakatit 


- `kryptolib.py`: vlastní Python knihovnu
- `main.py`: rozhraní s menu
- `notebook.ipynb`: demonstrační Jupyter notebook
- `Decrypted/`: složka s výsledky dešifrování
- `Encrypted/`, `ToCipher/`, `Cipher/`: složky s dalšími daty


In [7]:
try:
    from MH_decipher.kryptolib import score_by_dictionary
except ModuleNotFoundError:
    print("❌ Modul 'MH_decipher' nebyl nalezen. Definuji náhradní funkci 'score_by_dictionary'.")
    def score_by_dictionary(text, wordlist):
        # Fallback: return 0, or implement a simple scoring
        return 0

# Pokud není WORDLIST_SET definován, načti jej ze souboru
if 'WORDLIST_SET' not in globals():
    try:
        with open("wordlist.txt", encoding="utf-8") as f:
            WORDLIST_SET = set(word.strip().upper() for word in f)
    except FileNotFoundError:
        print("❌ Soubor 'wordlist.txt' nebyl nalezen. Ujistěte se, že existuje ve správném adresáři.")
        WORDLIST_SET = set()  # Inicializace prázdné množiny jako fallback

# Pokud není all_results definován, inicializuj prázdný seznam (nebo načti výsledky)
if 'all_results' not in globals():
    all_results = []

best_candidate, all_results = prolom_substitute_hybrid(
    encrypted, bigram_model_np, trigram_model_dict, iter_count=20000
)

key_found = best_candidate['key']
plaintext = best_candidate['plaintext']
score = best_candidate['score']

# Save the result to all_results
all_results.append({
    'key': key_found,
    'plaintext': plaintext,
    'score': score
})

print("===== ANALÝZA DOKONČENA =====")
print(f"\n🔓 Nalezený klíč: {key_found}")
print(f"📄 Dešifrovaný text: {decrypted}")
print(f"📈 Skóre: {float(score):.2f}")


❌ Modul 'MH_decipher' nebyl nalezen. Definuji náhradní funkci 'score_by_dictionary'.
❌ Soubor 'wordlist.txt' nebyl nalezen. Ujistěte se, že existuje ve správném adresáři.
===== ANALÝZA DOKONČENA =====

🔓 Nalezený klíč: GXWQMDOIJZNUEPCYLASBFVKH_TR
📄 Dešifrovaný text: BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS
📈 Skóre: 0.00


In [ ]:
import os

# Ensure 'best_candidate' is defined
if not all_results:
    print("❌ 'all_results' is empty. Cannot determine the best candidate.")
else:
    best_candidate = max(all_results, key=lambda r: score_by_dictionary(r['plaintext'], WORDLIST_SET))

    # Ensure the "Decrypted" directory exists
    os.makedirs("Decrypted", exist_ok=True)

    # Define the filename
    filename = "text_1000_sample_1"

    # Write the plaintext to a file
    with open(f"Decrypted/{filename}_plaintext.txt", "w", encoding="utf-8") as f:
        f.write(best_candidate['plaintext'])

    # Write the key to a file
    with open(f"Decrypted/{filename}_key.txt", "w", encoding="utf-8") as f:
        f.write(best_candidate['key'])

    print("✅ Uloženo do složky Decrypted/")


✅ Uloženo do složky Decrypted/
